In [32]:
import pandas as pd
import time

In [33]:
start=time.perf_counter()

In [34]:
from DatabaseCreator import DatabaseCreator
import CJDH_local_settings

#Run Database Creator
if __name__ == "__main__":
    db_creator = DatabaseCreator(db_settings=CJDH_local_settings.local_settings['FPL_Points_Predictor'])
    fpl_engine = db_creator.get_engine_for("fpl_data_analysis")

In [35]:
#add this info to a settings dict?
stats_current_gameweek = 9
picks_current_gameweek = 11
current_season = 20252026

In [77]:
print("When to use first TRIPLE CAPTAIN chip (gws1-19)")
view = db_creator.run_sql(f"""
    SELECT
        player_name_id,
        event,
        position,
        team_name,
        opp_team_name,
        xpoints
    FROM predictions
    WHERE in_my_team = 1
      AND event BETWEEN {picks_current_gameweek} AND 19
    ORDER BY xpoints DESC
""")
view.head(5)

When to use first TRIPLE CAPTAIN chip (gws1-19)


,player_name_id,event,position,team_name,opp_team_name,xpoints
0,Bruno Borges Fernandes,19.0,MID,Man Utd,Wolves,6.527842
1,Bruno Borges Fernandes,15.0,MID,Man Utd,Wolves,6.527842
2,Gabriel dos Santos MagalhÃ£es,16.0,DEF,Arsenal,Wolves,6.197677
3,Matheus Nunes,13.0,DEF,Man City,Leeds,6.036608
4,Bruno Borges Fernandes,14.0,MID,Man Utd,West Ham,5.979741


In [38]:
view = db_creator.run_sql(f"""
    SELECT
        player_name_id,
        in_my_team,
        position,
        team_name,
        ROUND(AVG(opp_team_elo),1) AS avg_opp_team_elo,
        ROUND(SUM(xpoints),2) AS total_xpoints,
        ROUND(AVG(mins_pred),1) AS avg_mins_pred,
        ROUND(AVG(xgp90_pred),3) AS avg_xgp90_pred,
        ROUND(AVG(xap90_pred),3) AS avg_xap90_pred,
        ROUND(AVG(xgcp90_pred),3) AS avg_xgcp90_pred
    FROM predictions
    WHERE (in_any_team = 0 OR in_my_team = 1)
      AND event BETWEEN {picks_current_gameweek} AND {picks_current_gameweek+5}
      -- AND position = 'FWD'
    GROUP BY player_name_id, position, team_name, in_my_team
    ORDER BY total_xpoints DESC
""")
view.head(20)

,player_name_id,in_my_team,position,team_name,avg_opp_team_elo,total_xpoints,avg_mins_pred,avg_xgp90_pred,avg_xap90_pred,avg_xgcp90_pred
0,Bruno Borges Fernandes,1,MID,Man Utd,1023.5,33.67,74.5,0.349,0.260,1.137
1,Gabriel dos Santos MagalhÃ£es,1,DEF,Arsenal,1053.8,33.13,83.6,0.123,0.073,0.879
2,Dominik Szoboszlai,1,MID,Liverpool,1045.8,29.95,82.3,0.203,0.244,1.254
3,Matheus Nunes,1,DEF,Man City,1063.1,29.21,79.6,0.051,0.064,0.943
4,Bruno GuimarÃ£es Rodriguez Moura,0,MID,Newcastle,1049.1,28.83,78.3,0.228,0.174,1.248
5,Amad Diallo,1,MID,Man Utd,1023.5,28.29,61.7,0.302,0.143,1.060
6,Dominic Calvert-Lewin,0,FWD,Leeds,1131.7,28.24,76.5,0.527,0.062,1.956
7,Dango Ouattara,1,MID,Bournemouth,1066.6,28.04,73.1,0.291,0.114,1.385
8,Carlos Henrique Casimiro,0,MID,Man Utd,1023.5,27.60,66.6,0.271,0.131,1.146
9,Yankuba Minteh,1,MID,Brighton,1084.4,27.58,80.3,0.196,0.173,1.474


In [69]:
print(f"\nBest available players to replace worst in my team over next 5 GWs: {picks_current_gameweek+1} to 38")#{picks_current_gameweek + 5}
view = db_creator.run_sql(f"""
    WITH 
    summed AS (
        SELECT 
            player_name_id,
            team_name,
            position,
            in_my_team,
            in_any_team,
            SUM(xpoints) AS total_xpoints
        FROM predictions
        WHERE event BETWEEN {picks_current_gameweek+1} AND 38 --{picks_current_gameweek + 5}
        GROUP BY player_name_id, team_name, position, in_my_team, in_any_team
    ),

    worst_gk AS (
        SELECT player_name_id, team_name, total_xpoints AS xpoints, in_my_team, position
        FROM summed
        WHERE in_my_team = 1
          AND position = 'GK'
        ORDER BY total_xpoints ASC
        LIMIT 1
    ),
    best_gk AS (
        SELECT player_name_id, team_name, total_xpoints AS xpoints, in_my_team, position
        FROM summed
        WHERE in_any_team = 0
          AND in_my_team = 0
          AND position = 'GK'
    ),

    worst_def AS (
        SELECT player_name_id, team_name, total_xpoints AS xpoints, in_my_team, position
        FROM summed
        WHERE in_my_team = 1
          AND position = 'DEF'
        ORDER BY total_xpoints ASC
        LIMIT 1
    ),
    best_def AS (
        SELECT player_name_id, team_name, total_xpoints AS xpoints, in_my_team, position
        FROM summed
        WHERE in_any_team = 0
          AND in_my_team = 0
          AND position = 'DEF'
    ),

    worst_mid AS (
        SELECT player_name_id, team_name, total_xpoints AS xpoints, in_my_team, position
        FROM summed
        WHERE in_my_team = 1
          AND position = 'MID'
        ORDER BY total_xpoints ASC
        LIMIT 1
    ),
    best_mid AS (
        SELECT player_name_id, team_name, total_xpoints AS xpoints, in_my_team, position
        FROM summed
        WHERE in_any_team = 0
          AND in_my_team = 0
          AND position = 'MID'
    ),

    worst_fwd AS (
        SELECT player_name_id, team_name, total_xpoints AS xpoints, in_my_team, position
        FROM summed
        WHERE in_my_team = 1
          AND position = 'FWD'
        ORDER BY total_xpoints ASC
        LIMIT 1
    ),
    best_fwd AS (
        SELECT player_name_id, team_name, total_xpoints AS xpoints, in_my_team, position
        FROM summed
        WHERE in_any_team = 0
          AND in_my_team = 0
          AND position = 'FWD'
    )

    SELECT 
        b.position,
        b.team_name AS best_team,
        b.player_name_id AS best_player_id,
        b.xpoints AS best_xpoints,
        w.player_name_id AS worst_player_id,
        w.xpoints AS worst_xpoints,
        (b.xpoints - w.xpoints) AS xpoints_diff
    FROM best_gk b
    CROSS JOIN worst_gk w

    UNION ALL

    SELECT 
        b.position,
        b.team_name AS best_team,
        b.player_name_id AS best_player_id,
        b.xpoints AS best_xpoints,
        w.player_name_id AS worst_player_id,
        w.xpoints AS worst_xpoints,
        (b.xpoints - w.xpoints) AS xpoints_diff
    FROM best_def b
    CROSS JOIN worst_def w

    UNION ALL

    SELECT 
        b.position,
        b.team_name AS best_team,
        b.player_name_id AS best_player_id,
        b.xpoints AS best_xpoints,
        w.player_name_id AS worst_player_id,
        w.xpoints AS worst_xpoints,
        (b.xpoints - w.xpoints) AS xpoints_diff
    FROM best_mid b
    CROSS JOIN worst_mid w

    UNION ALL

    SELECT 
        b.position,
        b.team_name AS best_team,
        b.player_name_id AS best_player_id,
        b.xpoints AS best_xpoints,
        w.player_name_id AS worst_player_id,
        w.xpoints AS worst_xpoints,
        (b.xpoints - w.xpoints) AS xpoints_diff
    FROM best_fwd b
    CROSS JOIN worst_fwd w
    ORDER BY xpoints_diff DESC
""")

view.head(20)



Best available players to replace worst in my team over next 5 GWs: 12 to 38


,position,best_team,best_player_id,best_xpoints,worst_player_id,worst_xpoints,xpoints_diff
0,FWD,Leeds,Dominic Calvert-Lewin,128.629269,Chris Wood,58.053704,70.575565
1,FWD,Burnley,Zian Flemming,110.738416,Chris Wood,58.053704,52.684712
2,FWD,Fulham,RaÃºl JimÃ©nez RodrÃ­guez,108.478926,Chris Wood,58.053704,50.425222
3,FWD,Leeds,Lukas Nmecha,77.283794,Chris Wood,58.053704,19.230090
4,FWD,Brighton,Charalampos Kostoulas,76.991104,Chris Wood,58.053704,18.937400
5,FWD,Nott'm Forest,Taiwo Awoniyi,76.122201,Chris Wood,58.053704,18.068497
6,FWD,Chelsea,Marc Guiu Paz,74.680804,Chris Wood,58.053704,16.627100
7,FWD,Crystal Palace,Eddie Nketiah,72.799304,Chris Wood,58.053704,14.745600
8,FWD,Everton,Norberto Bercique Gomes Betuncal,72.005107,Chris Wood,58.053704,13.951403
9,FWD,Brighton,Stefanos Tzimas,68.793692,Chris Wood,58.053704,10.739988


In [ ]:
# #Gt member player codes from the latest GW
# query = """
#         WITH
#         currentgw AS (
#             SELECT MAX(gw) as max_gw
#             FROM playergw)
#         SELECT *
#         FROM playergw
#         WHERE gw = currentgw
#         ;
#         """
# with fpl_engine.connect() as conn:
#     viewdf = pd.read_sql_query(query, conn)
# print(viewdf.shape)

In [ ]:
# #Query gets only player details from the latest GW
# query = """
#         ALTER TABLE player_details
#             RENAME COLUMN total_points TO total_points_gw;

#         WITH playergw AS (
#             SELECT *,
#             ROW_NUMBER() OVER(
#                     PARTITION BY
#                         season,
#                         code,
#                         event,
#                         fixture_id --Only one record per GW
#                     ORDER BY 
#                         now_datetime_utc DESC) as rn --Only select the latest date
#         FROM playergw
#         ),
#         currentgw AS (
#             SELECT MAX(current_gw) as max_gw
#             FROM playergw
#         )
        
#         SELECT *
#         FROM player_details
#         LEFT JOIN playergw
#             ON player_details.element = playergw.id
#             AND player_details.round = playergw.current_gw
#         CROSS JOIN currentgw
#         WHERE player_details.round = currentgw.max_gw
#             AND playergw.rn = 1
#         ;
#         """
# with engine.connect() as conn:
#     playerdf = pd.read_sql_query(query, conn)
# print(playerdf.shape)

# pd.pivot_table(playerdf, index=['id','web_name','team_name','round'],values='total_points_gw',aggfunc=np.sum).sort_values(by='total_points_gw', ascending=False)

In [ ]:
finish=time.perf_counter()
print(f'Finished in {round(((finish-start)/60),2)} minute(s)')
#10/11/25: Finished in 0.01 minute(s)

Finished in 0.01 minute(s)
